In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-08-26T18:21:12.165293-04:00

Python implementation: CPython
Python version       : 3.8.9
IPython version      : 8.4.0

Compiler    : Clang 13.1.6 (clang-1316.0.21.2.5)
OS          : Darwin
Release     : 21.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 20
Architecture: 64bit



In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import sklearn
import gc
import os

In [4]:
%watermark --iversions

pandas : 1.4.3
sklearn: 1.1.2
numpy  : 1.23.1



In [14]:
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')

In [11]:
train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
test.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
X = train[train.columns[1:]].values
X_test = test[test.columns[1:]].values
Y = train.label.values
Y_test = test.label.values

In [16]:
train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

(60000, 10)

In [20]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=300, validation_fraction=None, 
                                           random_state=137, max_depth=15)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.98375
Fitting fold 2
Fold accuracy: 0.9815833333333334
Fitting fold 3
Fold accuracy: 0.9815833333333334
Fitting fold 4
Fold accuracy: 0.9819166666666667
Fitting fold 5
Fold accuracy: 0.9784166666666667
0.98145
0.9825
CPU times: user 3h 36min 44s, sys: 4h 38min 42s, total: 8h 15min 26s
Wall time: 48min 56s


In [18]:
0.98375

0.98375

In [17]:
accuracy_score(test.label.values, preds)

0.9824

In [17]:
0.9824

0.9824

In [21]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=250, validation_fraction=None, 
                                           random_state=137, max_depth=15)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.9835833333333334
Fitting fold 2
Fold accuracy: 0.9811666666666666
Fitting fold 3
Fold accuracy: 0.9814166666666667
Fitting fold 4
Fold accuracy: 0.98175
Fitting fold 5
Fold accuracy: 0.9785
0.9812833333333333
0.9828
CPU times: user 3h 15min 1s, sys: 4h 5min 25s, total: 7h 20min 27s
Wall time: 44min 33s


In [22]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=300, validation_fraction=None, 
                                           random_state=137, max_depth=13)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.98375
Fitting fold 2
Fold accuracy: 0.9816666666666667
Fitting fold 3
Fold accuracy: 0.9815
Fitting fold 4
Fold accuracy: 0.9825833333333334
Fitting fold 5
Fold accuracy: 0.97875
0.98165
0.9831
CPU times: user 3h 41min 47s, sys: 4h 37min 55s, total: 8h 19min 43s
Wall time: 50min


In [23]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=275, validation_fraction=None, 
                                           random_state=137, max_depth=13)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.98375
Fitting fold 2
Fold accuracy: 0.9816666666666667
Fitting fold 3
Fold accuracy: 0.9813333333333333
Fitting fold 4
Fold accuracy: 0.9824166666666667
Fitting fold 5
Fold accuracy: 0.9783333333333334
0.9815
0.9829
CPU times: user 3h 28min 50s, sys: 4h 32min 42s, total: 8h 1min 32s
Wall time: 48min 21s


In [24]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=325, validation_fraction=None, 
                                           random_state=137, max_depth=13)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.984
Fitting fold 2
Fold accuracy: 0.98175
Fitting fold 3
Fold accuracy: 0.98175
Fitting fold 4
Fold accuracy: 0.9826666666666667
Fitting fold 5
Fold accuracy: 0.9786666666666667
0.9817666666666667
0.9829
CPU times: user 3h 59min 29s, sys: 4h 58min 42s, total: 8h 58min 11s
Wall time: 55min 9s


In [25]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=350, validation_fraction=None, 
                                           random_state=137, max_depth=13)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.98425
Fitting fold 2
Fold accuracy: 0.9816666666666667
Fitting fold 3
Fold accuracy: 0.9815833333333334
Fitting fold 4
Fold accuracy: 0.98275
Fitting fold 5
Fold accuracy: 0.9789166666666667
0.9818333333333333
0.9829
CPU times: user 4h 5min 34s, sys: 5h 3min 37s, total: 9h 9min 11s
Wall time: 57min 34s


In [26]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=375, validation_fraction=None, 
                                           random_state=137, max_depth=13)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.98425
Fitting fold 2
Fold accuracy: 0.9816666666666667
Fitting fold 3
Fold accuracy: 0.9815833333333334
Fitting fold 4
Fold accuracy: 0.98275
Fitting fold 5
Fold accuracy: 0.9789166666666667
0.9818333333333333
0.9829
CPU times: user 4h 2min 19s, sys: 4h 56min 20s, total: 8h 58min 39s
Wall time: 55min 45s


In [27]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=375, validation_fraction=None, 
                                           random_state=137, max_depth=12)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.9845
Fitting fold 2
Fold accuracy: 0.9814166666666667
Fitting fold 3
Fold accuracy: 0.982
Fitting fold 4
Fold accuracy: 0.9818333333333333
Fitting fold 5
Fold accuracy: 0.9785833333333334
0.9816666666666667
0.983
CPU times: user 4h 1min 17s, sys: 5h 21min 41s, total: 9h 22min 59s
Wall time: 58min 6s


In [28]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=400, validation_fraction=None, 
                                           random_state=137, max_depth=12)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.9845
Fitting fold 2
Fold accuracy: 0.9814166666666667
Fitting fold 3
Fold accuracy: 0.982
Fitting fold 4
Fold accuracy: 0.9818333333333333
Fitting fold 5
Fold accuracy: 0.9785833333333334
0.9816666666666667
0.983
CPU times: user 3h 54min 39s, sys: 4h 59min 46s, total: 8h 54min 26s
Wall time: 53min 16s


In [38]:
%%time

train_oof = np.zeros((X.shape[0], 10))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=800, validation_fraction=None, 
                                           random_state=137, learning_rate=0.05, 
                                           max_depth=20)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)
    train_oof[val_index] = val_pred
    print("Fold accuracy:", accuracy_score(val_target, np.argmax(val_pred, axis=1)))
    test_preds += model.predict_proba(X_test)/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

print(accuracy_score(Y, np.argmax(train_oof, axis=1)))

preds = np.argmax(test_preds, axis=1)

print(accuracy_score(Y_test, preds))

Fitting fold 1
Fold accuracy: 0.984
Fitting fold 2
Fold accuracy: 0.98175
Fitting fold 3
Fold accuracy: 0.9820833333333333
Fitting fold 4
Fold accuracy: 0.9821666666666666
Fitting fold 5
Fold accuracy: 0.97875
0.98175
0.9826
CPU times: user 7h 39min 10s, sys: 9h 28min 23s, total: 17h 7min 33s
Wall time: 1h 44min 26s


In [37]:
0.984

0.984